

#  节点表征学习（Node Representation Learning）

---

## 1 节点表征学习

节点表征学习（Node Embedding）是将图中每个节点映射为一个低维向量，使得向量之间的几何关系能反映原图中的**结构、属性或语义**关系。

目标：

$$
f: V \rightarrow \mathbb{R}^d \quad (d \ll |V|)
$$

---

###  目的

* 将**非欧结构化图数据**转换为可输入神经网络的向量形式
* 保留图中节点之间的**邻近性、结构相似性或角色相似性**

---

###  典型方法类别

| 方法类别  | 示例                      | 特点            |
| ----- | ----------------------- | ------------- |
| 矩阵分解类 | Laplacian Eigenmap、HOPE | 基于邻接/相似矩阵分解   |
| 游走类   | DeepWalk、Node2vec       | 基于路径采样的上下文建模  |
| GNN类  | GCN、GraphSAGE           | 聚合邻居特征，端到端可训练 |

---

## 2 基于矩阵分解的节点表征学习

---

该类方法核心思想：构造一个能表示**节点对关系强弱**的相似性矩阵 $S$，然后通过矩阵分解的方法获得嵌入。

$$
S \approx ZZ^\top \quad \text{或} \quad S \approx Z_1 Z_2^\top
$$

其中 $Z \in \mathbb{R}^{|V| \times d}$ 为节点嵌入矩阵。

---

## 2.1 低阶邻近关系（First-order Proximity）

---

###  定义

第一阶邻近表示图中**直接相连节点之间的关系强度**。

$$
S_{uv}^{(1)} = A_{uv}
$$

* $A$：邻接矩阵，表示节点间是否有边（或边的权重）

---

###  目标

保持图中**局部连接**，适合于表示强连接（例如好友/交易等一跳关系）

---

###  应用方法

* Laplacian Eigenmaps：通过最小化嵌入空间中的邻接点距离
* Graph Factorization：对 $A \approx ZZ^\top$ 进行低秩分解

---

## 2.2 高阶邻近关系（High-order Proximity）

---

### 定义

高阶邻近指图中**不直接连接**但在路径上结构相似的节点之间的关系。

* 二阶邻近：具有共同邻居的节点
* 三阶及以上邻近：在多跳路径中结构位置相似的节点

---

###  表示方式（常见高阶矩阵）

| 方法                    | 相似性矩阵 $S$                             |
| --------------------- | ------------------------------------- |
| Common Neighbors      | $A^2$                                 |
| Katz Index            | $S = \sum_{i=1}^{\infty} \beta^i A^i$ |
| SimRank               | 节点的相似性递归定义                            |
| Personalized PageRank | 见 2.4                                 |

---

###  应用方法

* **HOPE（High-Order Proximity preserved Embedding）**：对 Katz、Rooted PageRank 等矩阵进行近似分解
* **NetMF**：将 DeepWalk 隐式建模的 PMI 矩阵显式化再分解

---

## 2.3 PageRank

---

###  定义

PageRank 衡量一个节点在全图中**被访问的重要程度**，是全局结构中心性的典型表示。

$$
\pi = \alpha A^\top \pi + (1 - \alpha)\mathbf{1}/n
$$

* $\pi$：PageRank 向量
* $\alpha$：跳转概率（通常为 0.85）
* $A^\top$：转移矩阵（归一化邻接矩阵）

---

###  解释

* 模拟“随机浏览者”不断点击邻居链接
* 被更多重要节点指向的节点获得更高分数

---

###  在节点表征中的作用

* 可作为节点特征输入（PageRank score）
* 也可用作全图排序、链接预测等任务的标签

---

## 2.4 Rooted PageRank（个性化PageRank，PPR）

---

###  定义

Rooted PageRank 关注**从某个指定节点 $u$** 出发后回到自身或其邻居的概率，强调**个性化邻近性**：

$$
\pi_u = \alpha A^\top \pi_u + (1 - \alpha) e_u
$$

* $e_u$：从 $u$ 开始的一维独热向量
* $\pi_u$：表示从 $u$ 出发到所有节点的概率分布

---

###  特点

* 捕捉局部到中等范围的邻近信息
* 更适用于**异构图、非均匀结构图、图采样聚合**

---

###  应用方法

* **APP（Approximate Personalized Propagation）**：快速估算 Rooted PageRank 并用作表示
* **PPR-GNN**：将PPR近似结果作为 GNN 中的邻接关系

---

## 2.5 基于非对称邻近性拟合的节点表征学习

---

###  背景动机

许多真实图中存在**非对称连接关系**（例如：A 关注 B，但 B 不关注 A），导致**对称嵌入 $Z Z^\top$** 不足以表达结构信息。

---

###  非对称建模方式

采用双矩阵嵌入表示：

$$
S_{uv} \approx z_u^\top z'_v
$$

* $z_u$：源节点的向量
* $z'_v$：目标节点的向量（表示“被指向”的含义）

---

###  应用方法

* **HOPE**：对如 Katz Index、Rooted PageRank 等**非对称高阶邻近矩阵**进行**广义 SVD**分解

---

###  HOPE 示例：

对 Katz Index：

$$
S = \sum_{i=1}^{\infty} \beta^i A^i = (I - \beta A)^{-1} - I
$$

令：

$$
S \approx Z_1 Z_2^\top
$$

使用广义奇异值分解（G-SVD）得到嵌入 $Z_1, Z_2$，分别表示“起点角色”和“终点角色”

---

##  小结：矩阵分解 vs 游走类方法

| 方式                     | 优点         | 缺点             |
| ---------------------- | ---------- | -------------- |
| 矩阵分解（HOPE、NetMF）       | 理论清晰、结构性强  | 可扩展性弱、存储大      |
| 游走类（DeepWalk、Node2vec） | 可扩展、采样效率高  | 难以解释、不精确建模高阶关系 |
| PPR类（APP、PPR-GNN）      | 可压缩近似、高效传播 | 需要迭代求解或稀疏近似    |



#  3 从 Word2Vec 到 Node2Vec

图表示学习的早期经典方法，受到自然语言处理中的词嵌入模型（Word2Vec）启发。核心思想是：

> **将节点序列视作“句子”，将节点视作“词”，借助语言模型学习节点的上下文结构表示。**

---

## 3.1 One-Hot Representation 与 Distributed Representation（分布式表示）

---

###  One-Hot 表示（独热编码）

每个词或节点用一个长为 $|V|$ 的向量表示，仅在该元素位置为 1，其余为 0：

$$
\text{apple} \rightarrow [0,0,0,0,1,0,...,0]
$$

####  问题：

* 高维稀疏，难以度量相似性
* 无法表达语义、上下文信息
* 无泛化能力（相似节点之间独立编码）

---

###  Distributed Representation（分布式表示）

将词（或节点）表示为低维向量，能学习词与词之间的上下文关系：

$$
\text{apple} \rightarrow \mathbf{v}_{\text{apple}} \in \mathbb{R}^d, \quad d \ll |V|
$$

####  优点：

* 向量空间中，语义接近的词距离更近
* 可用于下游任务（分类、聚类、推荐等）
* 表示具备可学习性

---

## 3.2 Word2Vec：CBOW 与 Skip-gram

![](./img/2_1.png)

---

Word2Vec（Mikolov 等人提出）是一种**基于上下文建模词语向量**的模型。
本质上是：**用浅层神经网络学习词的嵌入向量**。

---

###  CBOW（Continuous Bag of Words）

* 给定上下文词 $\{w_{i-k}, ..., w_{i+k}\}$，预测中心词 $w_i$
* 假设上下文词顺序无关（Bag of Words）

$$
\text{maximize} \quad P(w_i \mid w_{i-k}, ..., w_{i+k})
$$

---

###  Skip-gram

* 给定中心词 $w_i$，预测上下文词
* 学习一个词在**多个上下文中出现的概率**

$$
\text{maximize} \quad \sum_{-k \le j \le k, j \ne 0} P(w_{i+j} \mid w_i)
$$

---

###  优化方法：

* **负采样（Negative Sampling）**
* **层次 Softmax（Hierarchical Softmax）**

---

## 3.3 图嵌入方法：DeepWalk 与 Node2Vec

---

###  3.3.1 DeepWalk（2014）

####  核心思想：

* 将图中的节点转化为“词”
* 用**随机游走（Random Walk）生成节点序列**，作为“句子”
* 然后将 Word2Vec Skip-gram 应用于这些序列中，学习节点向量

---

####  步骤：

1. 对图中的每个节点 $v$，多次执行长度为 $l$ 的随机游走，得到序列 $\mathcal{W}_v$
2. 在每条节点序列上使用 Skip-gram 构建节点上下文
3. 最终优化：

$$
\max_{\Phi} \sum_{v \in V} \sum_{u \in \mathcal{N}_w(v)} \log P(u \mid v)
$$

其中 $\Phi(v)$ 为节点 $v$ 的向量表示

---

####  特征：

| 特性   | 说明                 |
| ---- | ------------------ |
| 局部结构 | 保留随机游走路径下的邻近性      |
| 无监督  | 不依赖标签信息            |
| 缺点   | 无法灵活控制采样策略；仅使用结构信息 |

---

###  3.3.2 Node2Vec（2016）

####  改进动机：

DeepWalk 使用**纯随机游走**，无法灵活控制游走行为（例如 BFS/DFS）。

Node2Vec 提出**可调控的随机游走策略**，使得嵌入可以适应结构等价/社区相似等多种情形。

---

####  核心机制：**有偏随机游走（Biased Random Walk）**

* 两个参数：

  * $p$：返回参数（Return parameter）
  * $q$：进攻参数（In-out parameter）

---

####  控制机制：

* 如果想回到上一步节点 → 控制 $p$
* 如果想向远处扩展（结构等价） → 控制 $q$

| 走向类型    | 权重解释          |
| ------- | ------------- |
| 回到上一个节点 | $\frac{1}{p}$ |
| 邻近于当前节点 | 1             |
| 距离较远的节点 | $\frac{1}{q}$ |

---

####  特性：

| 特性    | 说明                        |
| ----- | ------------------------- |
| 通用性   | 包含 BFS（角色相似）与 DFS（社区相似）游走 |
| 可调节性  | 通过 $p, q$ 控制搜索策略          |
| 表示能力强 | 可统一表示不同结构位置的节点            |

---

###  DeepWalk vs Node2Vec 总结

| 模型       | 游走方式   | 可调性 | 保留信息类型    |
| -------- | ------ | --- | --------- |
| DeepWalk | 无偏随机游走 | 否   | 局部邻接      |
| Node2Vec | 有偏随机游走 | 是   | 局部 + 结构角色 |

---

## 3.4 随机游走与矩阵分解的等价性

---

###  背景

虽然 DeepWalk 是基于随机游走 + Skip-gram 实现的，但其**本质是对某个共现矩阵的矩阵分解**。

---

###  DeepWalk ≈ 隐式矩阵分解

参考论文：**“The Hidden Structure in DeepWalk” (Yang & Cohen, 2015)**

DeepWalk 实际上等价于对以下矩阵进行因式分解：

$$
M = \log\left( \frac{vol(G)}{T} \cdot \sum_{r=1}^{T} (D^{-1}A)^r \right) - \log b
$$

* $D^{-1}A$：转移概率矩阵
* $T$：窗口大小（即最大游走长度）
* $b$：负采样项

---

###  NetMF（2018）进一步理论化了这一观点：

将 DeepWalk 表示为：

$$
M = \log \left( \frac{1}{T} \sum_{r=1}^{T} P^r \cdot D^{-1} \right) \quad \text{(PMI matrix)}
$$

并提出了：

* 显式构建共现矩阵
* 然后使用 **SVD** 得到嵌入 $Z$

---

###  理解意义：

* DeepWalk 本质是在拟合一个**高阶邻近性矩阵**
* Node2Vec 类似在拟合不同权重的高阶路径矩阵
* 为后续的 HOPE、NetMF、LINE 等基于矩阵的建模方法提供理论依据

---

##  小结

| 方法       | 思路              | 优点         | 缺点            |
| -------- | --------------- | ---------- | ------------- |
| Word2Vec | 上下文共现建模         | 可扩展、有效     | 需要语料、不能处理结构数据 |
| DeepWalk | 节点序列 + Word2Vec | 利用结构，简单有效  | 不可调、局部感知      |
| Node2Vec | 有偏游走 + Word2Vec | 可调策略，适应不同图 | 参数调优复杂        |
| NetMF    | 显式矩阵建模          | 理论清晰       | 大图下存储成本高      |

---




#  4 使用深度学习进行节点表征学习

![](./img/2_2.png)

传统图嵌入方法如 DeepWalk/Node2Vec 虽然有效，但它们本质是“**固定图结构下的浅层模型**”。随着深度学习的发展，研究者逐步引入更强大的神经网络体系进行节点表示建模，具有：

* 高度非线性建模能力
* 支持端到端训练
* 能融合结构与属性信息
* 可扩展到异构图、动态图、复杂语义图等

---

## 4.1 基于深度自编码器的节点表征学习

---

### 核心思想

将图结构作为输入，利用自编码器网络（Autoencoder）压缩后重构邻接信息，实现节点嵌入。

---

### 模型结构

一个典型的图自编码器结构如下：

* **编码器（Encoder）**：从输入图结构中提取低维向量

  $$
  Z = f(X, A) \in \mathbb{R}^{|V| \times d}
  $$

* **解码器（Decoder）**：通过节点嵌入重构图结构（邻接矩阵）

  $$
  \hat{A}_{ij} = \sigma(z_i^\top z_j) \quad \text{或} \quad \hat{A} = Z Z^\top
  $$

* **损失函数**：最小化结构重构误差

  $$
  \mathcal{L}_{rec} = \| A - \hat{A} \|_F^2
  $$

---

### 代表模型

| 模型                                           | 特点                       |
| -------------------------------------------- | ------------------------ |
| **SDNE** (Structural Deep Network Embedding) | 利用深层自编码器捕捉非线性结构关系        |
| **DNGR** (Deep Neural Graph Representation)  | 基于 PPMI 的无监督预处理 + 深度自编码器 |
| **GAE/VGAE**                                 | 图卷积编码器 + 解码器（边重建）        |

---

### 优点

* 能建模非线性图结构
* 易于引入节点属性
* 可扩展为变分图自编码器（VGAE）

---

## 4.2 融合节点属性的节点表征学习

---

### 背景

多数实际图中不仅存在结构（边），还包含节点属性（如文本、特征向量等）。仅利用结构信息会丢失重要语义。

---

###  关键策略

将节点特征 $X \in \mathbb{R}^{|V| \times F}$ 融入嵌入生成中：

* 作为自编码器输入

  $$
  Z = f(X)
  $$

* 结合邻接结构

  $$
  Z = f(X, A)
  $$

* 或者融合方式：

  $$
  \text{Embedding} = \text{MLP}(X) + \text{GCN}(A, X)
  $$

---

###  应用方法

| 方法        | 融合方式      | 特点          |
| --------- | --------- | ----------- |
| GCN       | 卷积传播结构信息  | 结构 + 属性共建嵌入 |
| GraphSAGE | 聚合邻居属性    | 适合大图采样      |
| GAT       | 注意力选择邻居属性 | 加权融合邻居表示    |

---

###  表征目标

节点嵌入要同时捕捉：

* 本地结构（邻接关系）
* 节点语义（属性信息）
* 层级传播（高阶结构）

---

## 4.3 基于神经消息传递的节点表征学习（Neural Message Passing）

---

###  核心思想

神经消息传递机制（Message Passing Neural Network, MPNN）是一类**以“邻居聚合 + 自身更新”为核心**的深度图学习架构，被视为 GNN 模型的统一框架。

---

###  一般形式

设节点 $v$ 的表示为 $h_v^{(k)}$，第 $k$ 层传播如下：

$$
m_v^{(k)} = \text{AGGREGATE}^{(k)}\left( \{ h_u^{(k-1)} \mid u \in \mathcal{N}(v) \} \right)
$$

$$
h_v^{(k)} = \text{UPDATE}^{(k)}\left( h_v^{(k-1)}, m_v^{(k)} \right)
$$

* **AGGREGATE**：聚合邻居节点信息（如平均、加权和、最大值等）
* **UPDATE**：融合节点自身状态与邻居信息（如 MLP、GRU）

---

###  典型模型框架

| 模型        | 聚合方式                    | 特点             |
| --------- | ----------------------- | -------------- |
| GCN       | 卷积归一化邻接                 | 简洁高效，端到端可训练    |
| GraphSAGE | 聚合函数可选（mean, max, LSTM） | 支持大图采样训练       |
| GAT       | 注意力加权邻居                 | 结构可解释性好        |
| GIN       | 多层 MLP 聚合 + injective映射 | 表达力强，接近理论最优判别力 |
| MPNN      | 抽象消息传递模型                | 统一视角，包括以上模型    |

---

###  对比其他方式

| 方法          | 是否端到端 | 是否使用属性 | 是否建模结构 |
| ----------- | ----- | ------ | ------ |
| DeepWalk    | 否     | 否      | 是      |
| Node2Vec    | 否     | 否      | 是      |
| Autoencoder | 是     | 可选     | 是      |
| GNN（MPNN）   | ✅     | ✅      | ✅      |

---

###  总结

神经消息传递机制构成现代图神经网络的基本构件，其设计要点包括：

* **如何选择邻居聚合方式**
* **如何设计节点更新函数**
* **如何处理图的异质性、方向性、动态性**



---

## 小结

| 子模块      | 方法代表            | 技术路径         | 适用性      |
| -------- | --------------- | ------------ | -------- |
| 图自编码器    | SDNE, DNGR, GAE | 基于结构的编码-解码   | 表征重构、边预测 |
| 属性融合嵌入   | GCN, GraphSAGE  | 属性 + 结构联合建模  | 分类、聚类    |
| 消息传递 GNN | MPNN, GAT, GIN  | 聚合 + 更新 + 迭代 | 泛化能力强    |


